# Calibrate with particles 

The idea is to run PyPTV as usual, and check the box "Use only 4 frames". The result will be in the /res folder with only quadruplets as 3D and the respective indices of 2D targets per image

If we read this dataset into the proper format, we can now reproject every 3D point in rt_is back into the image and then optimize calibration with disparity between the position of the target as detected and the reprojected center. 

In [ ]:
# -*- coding: utf-8 -*-
# copy of https://github.com/alexlib/pbi/blob/master/ptv/shake.py
"""
BOOM shake shake shake the room!!!

Fine-tune calibration using the "shaking" method of comparing 3D positions 
obtained with existing calibration to their 2D projections. It's a kind of a 
feedback step over the normal calibration with known points.

Created on Sun Jan 31 13:42:18 2016

@author: Yosef Meller
"""
import numpy as np

from optv.orientation import full_calibration
from optv.tracking_framebuf import TargetArray, Frame
from pyptv.ptv import full_scipy_calibration


import os
from pathlib import Path

present_folder = Path.cwd()

working_folder = Path("/home/user/Documents/repos/test_cavity")
par_path = working_folder / "parameters"
working_folder.exists(), par_path.exists()

from pyptv.ptv import py_start_proc_c
from pyptv.parameters import OrientParams


# we work inside the working folder, all the other paths are relative to this 
num_cams = 4
os.chdir(working_folder)
cpar, spar, vpar, track_par, tpar, calibs, epar = py_start_proc_c(num_cams)
assert cpar.get_num_cams() == num_cams

targ_files = [spar.get_img_base_name(c).decode().split('%d')[0].encode() for c in \
    range(num_cams)]

print(targ_files)


# recognized names for the flags:
NAMES = ["cc", "xh", "yh", "k1", "k2", "k3", "p1", "p2", "scale", "shear"]
op = OrientParams()
op.read()
flags = [name for name in NAMES if getattr(op, name) == 1]

print(flags)


[b'img_3/cam1.', b'img_3/cam2.', b'img_3/cam3.', b'img_3/cam4.']
[]


In [2]:
def backup_ori_files(cpar):
    """ backup ORI/ADDPAR files to the backup_cal directory """
    import shutil 
    for i_cam in range(cpar.get_num_cams()):
        f = cpar.get_cal_img_base_name(i_cam).decode()
        print(f"Backing up {f}.ori")
        shutil.copyfile(f + ".ori", f + ".ori.bck")
        shutil.copyfile(f + ".addpar", f + ".addpar.bck")


# Backup is the first thing to do 
backup_ori_files(cpar)

Backing up cal/cam1.tif.ori
Backing up cal/cam2.tif.ori
Backing up cal/cam3.tif.ori
Backing up cal/cam4.tif.ori


In [3]:
print("Starting from: calibration")
for cam in range(num_cams):
    print(f"{cam=} {calibs[cam].get_pos()}, {calibs[cam].get_angles()}")
    

Starting from: calibration
cam=0 [  78.11540025   15.92351745 -570.78991499], [-56.53585681   2.98297501  56.53131934]
cam=1 [-118.04855126   27.22394654 -578.45047078], [ 0.03304577 -2.93444417 -0.01542235]
cam=2 [-111.5094956    72.49307514  584.63772122], [-0.11023317 -0.19965175 -0.02763281]
cam=3 [124.39034813  68.30563702 573.54174179], [-0.11962148  0.23596931  0.00940686]


In [4]:
# Iterate over frames, loading the big lists of 3D positions and 
# respective detections.
all_known = []
all_detected = [[] for c in range(cpar.get_num_cams())]

for frm_num in range(spar.get_first(), spar.get_last() + 1): # all frames for now, think of skipping some
    frame = Frame(cpar.get_num_cams(), 
        corres_file_base = ('res/rt_is').encode(), 
        linkage_file_base= ('res/ptv_is').encode(),
        target_file_base = targ_files, 
        frame_num = frm_num)
        
    all_known.append(frame.positions())
    for cam in range(cpar.get_num_cams()):
        all_detected[cam].append(frame.target_positions_for_camera(cam))

# Make into the format needed for full_calibration.
all_known = np.vstack(all_known)


Can't open ascii file: img_3/cam1._targets


In [5]:

# Calibrate each camera accordingly.
for cam in range(num_cams):
    detects = np.vstack(all_detected[cam])
    assert detects.shape[0] == all_known.shape[0]
    
    have_targets = ~np.isnan(detects[:,0])
    used_detects = detects[have_targets,:]
    used_known = all_known[have_targets,:]
    
    targs = TargetArray(len(used_detects))
    
    for tix in range(len(used_detects)):
        targ = targs[tix]
        targ.set_pnr(tix)
        targ.set_pos(used_detects[tix])
    
    # residuals = full_calibration(calibs[cam], used_known, targs, cpar)


    try:
        residuals, targ_ix, err_est = full_calibration(
            calibs[cam],
            used_known,
            targs,
            cpar,
            flags=[],
        )
        print(f"After full calibration, {np.sum(residuals**2)}")

        print(("Camera %d" % (cam + 1)))
        print((calibs[cam].get_pos()))
        print((calibs[cam].get_angles()))

    except Exception as e:
        print(f"Error in full_calibration: {e}, run Scipy.optimize")
        continue


    # else:
    #     if args.output is None:
    #         ori = cal_args[cam]['ori_file']
    #         distort = cal_args[cam]['addpar_file']
    #     else:
    #         ori = args.output % (cam + 1) + '.ori'
    #         distort = args.output % (cam + 1) + '.addpar'
        
    #     calibs[cam].write(ori.encode(), distort.encode())

After full calibration, 4238.65116277513
Camera 1
[ 117.92618753  -96.22618625 -661.83355869]
[-56.69999735   2.92738529  56.50535602]
Error in full_calibration: Orientation iteration failed, need better setup., run Scipy.optimize
After full calibration, 4761.482349732552
Camera 3
[-190.78002628   84.71599112  655.67206225]
[-0.13408924 -0.31725406 -0.06968912]
After full calibration, 4526.47521830534
Camera 4
[ 97.33291204 148.02769446 666.61990242]
[-0.24134346  0.13825767 -0.01346956]


In [6]:

# Calibrate each camera accordingly.
for cam in range(num_cams):
    detects = np.vstack(all_detected[cam])
    assert detects.shape[0] == all_known.shape[0]
    
    have_targets = ~np.isnan(detects[:,0])
    used_detects = detects[have_targets,:]
    used_known = all_known[have_targets,:]
    
    targs = TargetArray(len(used_detects))
    
    for tix in range(len(used_detects)):
        targ = targs[tix]
        targ.set_pnr(tix)
        targ.set_pos(used_detects[tix])
    
    # residuals = full_calibration(calibs[cam], used_known, targs, cpar)


    try:
        residuals, targ_ix, err_est = full_calibration(
            calibs[cam],
            used_known,
            targs,
            cpar,
            flags=flags,
        )
        print(f"After full calibration, {np.sum(residuals**2)}")

        print(("Camera %d" % (cam + 1)))
        print((calibs[cam].get_pos()))
        print((calibs[cam].get_angles()))

    except Exception as e:
        print(f"Error in full_calibration: {e}, run Scipy.optimize")
        continue


After full calibration, 4238.651162775129
Camera 1
[ 117.92616027  -96.22619204 -661.83356428]
[-56.69999736   2.92738534  56.50535602]
Error in full_calibration: Orientation iteration failed, need better setup., run Scipy.optimize
After full calibration, 4761.482349732552
Camera 3
[-190.78009964   84.71595015  655.67203841]
[-0.13408918 -0.31725419 -0.06968912]
After full calibration, 4526.47521830534
Camera 4
[ 97.33294504 148.02771737 666.61989517]
[-0.24134349  0.13825772 -0.01346955]


In [7]:

# Calibrate each camera accordingly.
for cam in range(num_cams):
    detects = np.vstack(all_detected[cam])
    assert detects.shape[0] == all_known.shape[0]
    
    have_targets = ~np.isnan(detects[:,0])
    used_detects = detects[have_targets,:]
    used_known = all_known[have_targets,:]
    
    targs = TargetArray(len(used_detects))
    
    for tix in range(len(used_detects)):
        targ = targs[tix]
        targ.set_pnr(tix)
        targ.set_pos(used_detects[tix])
    


    residuals = full_scipy_calibration(
        calibs[cam],
        used_known,
        targs,
        cpar,
        flags=flags
    )
    print(f"After scipy full calibration, {np.sum(residuals**2)}")
    
    print(("Camera %d" % (cam + 1)))
    print((calibs[cam].get_pos()))
    print((calibs[cam].get_angles()))

# targ_ix = [t.pnr() for t in targs if t.pnr() != -999]
# targ_ix = np.arange(len(all_detected))

# save the results from calibs[cam]
# _write_ori(i_cam, addpar_flag=True)

After scipy full calibration, 2943.5077519271726
Camera 1
[ 117.92616027  -96.22619204 -661.83356428]
[-56.69999736   2.92738534  56.50535602]
After scipy full calibration, 3780.3093437748353
Camera 2
[-118.04855126   27.22394654 -578.45047078]
[ 0.03304577 -2.93444417 -0.01542235]
After scipy full calibration, 3306.58496509205
Camera 3
[-190.78009964   84.71595015  655.67203841]
[-0.13408918 -0.31725419 -0.06968912]
After scipy full calibration, 3143.385568267597
Camera 4
[ 97.33294504 148.02771737 666.61989517]
[-0.24134349  0.13825772 -0.01346955]
